In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver

In [10]:
url = 'https://www.moel.go.kr/info/defaulter/defaulterList.do'
resp = requests.get(url)

In [11]:
import chromedriver_autoinstaller


chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

'/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/chromedriver_autoinstaller/106/chromedriver'

In [12]:
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome()

In [13]:
driver.get(url)

In [14]:
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
select_menu=(driver.find_elements(By.NAME, 'pageUnit'))


In [15]:
select = Select(select_menu[0])

In [16]:
from selenium.webdriver.common.keys import Keys
select.select_by_value('100')
driver.find_elements(By.CLASS_NAME, 'btn_arrears')[0].click()


In [17]:
pages = driver.find_elements(By.CLASS_NAME, 'page')

In [45]:
df = pd.DataFrame(columns=['name', 'age', 'CMP_NM', 'unpaid_amount'])

In [52]:
columns = ['name', 'age', 'CMP_NM', 'unpaid_amount']
for i in range(8):
    if i > 0:
        xp = driver.find_elements(By.XPATH, f'//*[@id="frm"]/div[5]/a[{i}]')
        driver.implicitly_wait(2)
        try:
            xp[0].click()
        except:
            break
        driver.implicitly_wait(1)
    for j in range(1, 101):
        try:
            row = driver.find_elements(By.XPATH, f'//*[@id="frm"]/div[4]/div/table/tbody/tr[{j}]')
            str_row = str(row[0].text)
            tokens = str_row.split(' ')
            len_tokens = len(tokens)
            kor_name = tokens[0]
            kor_age = tokens[1]
            kor_company_name = tokens[2]
            unpaid_amount = tokens[len_tokens - 1]
            df_row = pd.DataFrame(np.zeros([1, len(columns)]), columns=columns)
            df_row['name'] = kor_name
            df_row['age'] = kor_age
            df_row['CMP_NM'] = kor_company_name
            df_row['unpaid_amount'] = unpaid_amount
            df = pd.concat([df, df_row], ignore_index=True)
        except:
            pass

In [53]:
df.to_csv('unpaid_company.csv')